In [1]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet

# TODO: Load traffic signs data.
training_file = "train.p"
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']





In [2]:

# TODO: Split data into training and validation sets.

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=832289)
train_len = len(X_train)
val_len = len(X_val)


print("train data set size:" ,train_len)
print("test data set size:",val_len)

train data set size: 33327
test data set size: 5882


In [3]:
# TODO: Define placeholders and resize operation.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))



In [4]:
# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)


In [5]:
# TODO: Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], 43)  # use this shape for the weight matrix
fc8_W = tf.Variable(tf.truncated_normal(shape=shape,mean=0,stddev=0.1))
fc8_b = tf.Variable(tf.zeros(43))
logits = tf.add(tf.matmul(fc7,fc8_W),fc8_b)

In [10]:
# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(tf.float32)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01,beta1=0.8, beta2=0.9)# beta for learning rate decay tech
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [11]:

# TODO: Train and evaluate the feature extraction model.

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_lost = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        loss,accuracy = sess.run([loss_operation,accuracy_operation], feed_dict={x: batch_x, y: batch_y,keep_prob: 1})
        total_accuracy += (accuracy * len(batch_x))
        total_lost += (loss*len(batch_x))
    t_lost = total_lost/num_examples
    t_accu = total_accuracy / num_examples
    return t_lost,t_accu


In [ ]:
#Traing the model
from sklearn.utils import shuffle
EPOCHS = 10
BATCH_SIZE = 128

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, tf.train.latest_checkpoint('.'))
    num_examples = len(X_train)
    
    
    training_loss_history = []
    validation_loss_history = []
    training_accuracy_history = []
    validation_accuracy_history = []
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob: 0.5})
            
            
        # Log accuracy and loss every epoch
        training_loss, training_accuracy = evaluate(X_train,y_train)
        training_loss_history.append(training_loss)
        training_accuracy_history.append(training_accuracy)
        
        validation_loss, validation_accuracy = evaluate(X_val, y_val)
        validation_loss_history.append(validation_loss)
        validation_accuracy_history.append(validation_accuracy)

        print("EPOCH {} ...".format(i+1))
        print("Training loss = {:.3f}".format(training_loss))
        print("Training accuracy = {:.3f}".format(training_accuracy))
        print("Validation loss = {:.3f}".format(validation_loss))
        print("Validation accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    # Evaluate on the test data
    test_loss, test_acc = evaluate(X_test, y_test)
    print("Test loss = {:.3f}".format(test_loss))
    print("Test accuracy = {:.3f}".format(test_acc))

    loss_plot = plt.subplot(2,1,1)
    loss_plot.set_title('Loss')
    loss_plot.plot(training_loss_history, 'r', label='Training Loss')
    loss_plot.plot(validation_loss_history, 'b', label='Validation Loss')
    plt.savefig('Report/loss.png')
    loss_plot.set_xlim([0, EPOCHS])
    loss_plot.legend(loc=4)
    acc_plot = plt.subplot(2,1,2)
    acc_plot.set_title('Accuracy')
    acc_plot.plot(training_accuracy_history,'r', label='Training Accuracy')
    acc_plot.plot(validation_accuracy_history, 'b', label='Validation Accuracy')
    acc_plot.set_ylim([0, 1.0])
    acc_plot.set_xlim([0, EPOCHS])
    acc_plot.legend(loc=4)
    plt.tight_layout()
    plt.show()
    
    
    saver.save(sess, './mylenet')
    print("Model saved")

Training...

EPOCH 1 ...
Training loss = 0.609
Training accuracy = 0.860
Validation loss = 0.814
Validation accuracy = 0.832

EPOCH 2 ...
Training loss = 0.389
Training accuracy = 0.906
Validation loss = 0.660
Validation accuracy = 0.874

EPOCH 3 ...
Training loss = 0.410
Training accuracy = 0.919
Validation loss = 0.660
Validation accuracy = 0.891

EPOCH 4 ...
Training loss = 0.423
Training accuracy = 0.920
Validation loss = 0.743
Validation accuracy = 0.885

EPOCH 5 ...
Training loss = 0.486
Training accuracy = 0.915
Validation loss = 0.768
Validation accuracy = 0.887

EPOCH 6 ...
Training loss = 0.324
Training accuracy = 0.940
Validation loss = 0.697
Validation accuracy = 0.905

EPOCH 7 ...
Training loss = 0.207
Training accuracy = 0.955
Validation loss = 0.512
Validation accuracy = 0.923

EPOCH 8 ...
Training loss = 0.162
Training accuracy = 0.967
Validation loss = 0.466
Validation accuracy = 0.935

EPOCH 9 ...
Training loss = 0.118
Training accuracy = 0.976
Validation loss = 0.418